In [194]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
from selenium.webdriver import ActionChains


#### Simulate selecting date on the date picker of the webpage

In [195]:
month2num = dict()
num2month = dict()
months = ['January', 'February','March','April','May','June','July','August','September','October','November','December']
months_abb = ['Jan', 'Feb', 'Mar','Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


nums = range(1,13)
for i in range(12):
    month2num[months_abb[i]] = nums[i]
    num2month[nums[i]] = months_abb[i]

def UpdateCalPage(calendar_dates, input_dates):
    cal_day, cal_month, cal_year = calendar_dates.split('/') 
    input_day, input_month, input_year = input_dates.split('/') 
    click_times = 12*(int(input_year) - int(cal_year)) + int(input_month) - int(cal_month)
    print(click_times)
    if click_times > 0:
        for i in range(click_times):
            time.sleep(2)
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > span > div.body_text > div > div.rsdc-body > div.rsdc-next.rsdc-nav.ui_icon.single-chevron-right-circle').click()
            print('click')
    elif click_times < 0:
        for i in range(abs(click_times)):
            time.sleep(2)
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > span > div.body_text > div > div.rsdc-body > div.rsdc-prev.rsdc-nav.ui_icon.single-chevron-left-circle').click()
            print('click')
    else:
        pass
    
def calendar(wait, driver, dep_date, ret_date):
    cal_dates = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="BODY_BLOCK_JQUERY_REFLOW"]/span/div[3]/div/div[2]/div[3]/span/span[1]'))).text
    print(cal_dates)
    cal_month, cal_year = cal_dates.split()
    cal_month = month2num[cal_month]
    cal_month = str(cal_month)
    if(len(cal_month) == 1):
        cal_month = '0' + str(cal_month)
    cal_dates = '10/' + cal_month + '/'+cal_year 
    print(cal_dates)
    
    # select departure date
    UpdateCalPage(cal_dates, dep_date)
    dep_day, dep_month, dep_year = dep_date.split('/') 
    tmp = int(dep_month) - 1
    if len(dep_month) == 1:
        dep_month = '0' + str(tmp)
    else:
        dep_month = str(tmp)
    
    pattern_dep = dep_year + '-' + dep_month + '-' + dep_day
    pattern_dep = 'span[data-date ="' + pattern_dep +'"]'  
    print(pattern_dep)
    click_dep = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, pattern_dep)))
    ActionChains(driver).move_to_element(click_dep).click(click_dep).perform()
    print('click departure date')

    # select return date
    
    time.sleep(5)
    UpdateCalPage(dep_date, ret_date)
    ret_day, ret_month, ret_year = ret_date.split('/') 
    tmp = int(ret_month) - 1
    if len(ret_month) == 1:
        ret_month = '0' + str(tmp)
    else:
        ret_month = str(tmp)
    pattern_ret = ret_year + '-' + ret_month + '-' + ret_day
    pattern_ret = "span[data-date ='" + pattern_ret +"']"  
    print(pattern_ret)
    click_ret = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, pattern_ret)))
    ActionChains(driver).move_to_element(click_ret).click(click_ret).perform()
    print('click departure date')
    print('click return date')
    

def refresh_page(driver, max_iter = 3):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    i = 0
    while('flights-search-results-itinerary-card-FlightItineraryCard__container--3yCEY' in soup.text):
        driver.refresh()
        time.sleep(10)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        i += 1
        if i > max_iter:
            return 
        
    return soup


In [196]:
# step1: get into the target url
domain = 'https://www.tripadvisor.com'
target_url = 'https://www.tripadvisor.com/CheapFlightsHome'

opt = webdriver.ChromeOptions()
# opt.set_headless()
opt.add_experimental_option('excludeSwitches', ['enable-automation']) # Turn on developer mode
driver = webdriver.Chrome('/Users/haorongzhu/Downloads/chromedriver', options=opt)

wait = WebDriverWait(driver, 40)
driver.get(target_url)

orig_str = 'Manchester'
dest_str = 'London'
dep_date = '12/12/2020'
ret_date = '16/12/2020'

# step2: input departure and destination

print('You want to go to {}!'.format(dest_str))
from_box = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="taplc_trip_search_home_flights_0"]/div[2]/div/span/div[1]/div[2]/div[1]/div/div[1]/input[2]')))
from_box.click()
from_box.clear()
from_box.send_keys(orig_str)
try:
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="taplc_trip_search_home_flights_0"]/div[2]/div/span/div[1]/div[2]/div[1]/div/div[1]/div/ul/li[1]'))).click()
except:
    pass


to_box = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="taplc_trip_search_home_flights_0"]/div[2]/div/span/div[1]/div[2]/div[1]/div/div[2]/input[2]')))
to_box.click()
to_box.clear()
to_box.send_keys(dest_str)
try:
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="taplc_trip_search_home_flights_0"]/div[2]/div/span/div[1]/div[2]/div[1]/div/div[2]/div/ul/li[1]'))).click()
except:
    pass

# step3: select dates to view deals
c = driver.find_element_by_xpath('//*[@id="rt_ui_picker"]/span[1]')
ActionChains(driver).move_to_element(c).click(c).perform()
print('click may 1')
time.sleep(3)
calendar(wait, driver, dep_date, ret_date)

search_btn =  wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="taplc_trip_search_home_flights_0"]/div[2]/div/span/div[1]/div[2]/div[4]/button')))
search_btn.click()


# processing popup
time.sleep(15)
soup = refresh_page(driver)

You want to go to London!
click may 1
May 2020
10/05/2020
7
click
click
click
click
click
click
click
span[data-date ="2020-11-12"]
click departure date
0
span[data-date ='2020-11-16']
click departure date
click return date


In [197]:
# Step 4 Initialization
fieldnames = [
                'inb_time', 'inb_airport', 'inb_flight', 'inb_duration', 'inb_stop_times', 'inb_stop_airport',
                'outb_time', 'outb_airport', 'inb_flight', 'outb_duration', 'outb_stop_times', 'outb_stop_airport',
                'price', 'flycore'
             ]
df_flight = pd.DataFrame(columns = fieldnames)

In [198]:
# Step 5 scrap all info we need
each_plan = soup.find_all('div', 
                          {'class' : "flights-search-results-itinerary-card-FlightItineraryCard__container--3yCEY"})
for idx, ep in enumerate(each_plan):
    price = ep.find_all('span', 
                        {'class' : "flights-search-results-itinerary-card-components-Title__price--xfl94"})[0].text
    
    inbound_time = ep.find_all('div', 
                               {'class' : "flights-search-results-itinerary-card-components-OneWayInfo__odTime--mp9Dl"})[0].text
    
    outbound_time = ep.find_all('div', 
                                {'class' : "flights-search-results-itinerary-card-components-OneWayInfo__odTime--mp9Dl"})[1].text
    
    inb_airport, inb_flight = (ep.find_all('div', 
                                           {'class' : "flights-search-results-itinerary-card-components-OneWayInfo__odAirline--EXFOh"})[0].text).split(',')
    inb_flight = inb_flight.strip()
    
    outb_airport, outb_flight = (ep.find_all('div', 
                                             {'class' : "flights-search-results-itinerary-card-components-OneWayInfo__odAirline--EXFOh"})[1].text).split(',')
    outb_flight = outb_flight.strip()
    
    inb_duration = each_plan[0].find_all('div', 
                                         {'class' : "flights-search-results-itinerary-card-components-OneWayInfo__duration--3ITx_"})[0].text
    
    outb_duration = each_plan[0].find_all('div', 
                                          {'class' : "flights-search-results-itinerary-card-components-OneWayInfo__duration--3ITx_"})[1].text
    
    inb_stop = each_plan[0].find_all('div', 
                                     {'class' : "flights-search-results-itinerary-card-components-OneWayInfo__stops--1wF5n"})[0].text
    
    outb_stop = each_plan[0].find_all('div', 
                                      {'class' : "flights-search-results-itinerary-card-components-OneWayInfo__stops--1wF5n"})[1].text

    if ',' in inb_stop:
        inb_stop_times, inb_stop_airport = inb_stop.split(',')
        inb_stop_airport = inb_stop_airport.strip()
    else: 
        inb_stop_times = inb_stop
        inb_stop_airport = ''


    if ',' in outb_stop:
        outb_stop_times, outb_stop_airport = outb_stop.split(',')
        outb_stop_airport = outb_stop_airport.strip()
    else: 
        outb_stop_times = outb_stop
        outb_stop_airport = ''
    
    fly_score = ep.find_all('div', {'class' : "flights-search-results-itinerary-card-components-FlyScoreInfo__container--3sFJZ"})[0].text[9:]
    
    # storage
    df_flight.loc[idx] = [
        inbound_time, inb_airport, inb_flight, inb_duration, inb_stop_times, inb_stop_airport,
        outbound_time, outb_airport, outb_flight, outb_duration, outb_stop_times, outb_stop_airport,
        price, fly_score
    ]


In [199]:
df_flight

,inb_time,inb_airport,inb_flight,inb_duration,inb_stop_times,inb_stop_airport,outb_time,outb_airport,inb_flight,outb_duration,outb_stop_times,outb_stop_airport,price,flycore
0,5:10 PM - 6:10 PM,MAN - LHR,British Airways,1h 0m,Nonstop,,8:00 AM - 9:00 AM,LHR - MAN,British Airways,1h 0m,Nonstop,,£151,7.3
1,3:05 PM - 4:10 PM,MAN - LHR,British Airways,1h 0m,Nonstop,,8:00 AM - 9:00 AM,LHR - MAN,British Airways,1h 0m,Nonstop,,£151,7.3
2,5:10 PM - 6:10 PM,MAN - LHR,British Airways,1h 0m,Nonstop,,1:10 PM - 2:15 PM,LHR - MAN,British Airways,1h 0m,Nonstop,,£151,6.9
3,5:10 PM - 6:10 PM,MAN - LHR,British Airways,1h 0m,Nonstop,,3:15 PM - 4:20 PM,LHR - MAN,British Airways,1h 0m,Nonstop,,£151,7.3
4,5:10 PM - 6:10 PM,MAN - LHR,British Airways,1h 0m,Nonstop,,6:25 PM - 7:30 PM,LHR - MAN,British Airways,1h 0m,Nonstop,,£151,7.3
5,5:10 PM - 6:10 PM,MAN - LHR,British Airways,1h 0m,Nonstop,,5:30 PM - 6:35 PM,LHR - MAN,British Airways,1h 0m,Nonstop,,£151,7.3
6,12:00 PM - 1:10 PM,MAN - LHR,British Airways,1h 0m,Nonstop,,8:00 AM - 9:00 AM,LHR - MAN,British Airways,1h 0m,Nonstop,,£151,7.3
7,3:05 PM - 4:10 PM,MAN - LHR,British Airways,1h 0m,Nonstop,,5:30 PM - 6:35 PM,LHR - MAN,British Airways,1h 0m,Nonstop,,£151,7.3
8,3:05 PM - 4:10 PM,MAN - LHR,British Airways,1h 0m,Nonstop,,3:15 PM - 4:20 PM,LHR - MAN,British Airways,1h 0m,Nonstop,,£151,7.3
9,3:05 PM - 4:10 PM,MAN - LHR,British Airways,1h 0m,Nonstop,,6:25 PM - 7:30 PM,LHR - MAN,British Airways,1h 0m,Nonstop,,£151,7.3


In [200]:
df_flight.to_csv('flight_info.csv')